### 한강대교 유량 결측치 처리
1. 사용할 컬럼: 자기 자신 -> 청담대교 수위 -> 행주대교 유량 -> 잠수교 수위 -> 청담대교 유량 -> 광진교 유량 -> 한강대교 수위 -> 강화대교 조위 -> 방류량 -> 강수량... -> 행주대교 수위
2. 그런데 이 순서는 모델의 행동을 설명하는 것이지 인과관계를 반드시 보장한다고 할 수는 없음...

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

import time
from glob import glob
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras import backend as K
# from keras.utils import plot_model

from keras.layers import Activation, Dense, LSTM, GRU, Bidirectional, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization, TimeDistributed, Input, Reshape, RepeatVector, Concatenate, GlobalMaxPooling1D 
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D


#GPU 자원이 부족
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import os
os.chdir('/content/drive/MyDrive/신입기수 프로젝트/Dataset/')

In [4]:
# 결측치 처리한 수위4개+유량1개 데이터셋 가져오기
raw_data = pd.read_csv('rawdata.csv', encoding = 'UTF-8')

# 6912개 수위데이터 + 한강대교_수위 데이터셋 가져오기
newdata5 = pd.read_csv('Copy of 0827 광진행주유량수위x4.csv', encoding = 'cp949')

# 광진교 수위 컬럼 추가
raw_data = pd.concat([raw_data['방류량'], newdata5['광진교_수위']], axis = 1)

In [5]:
raw_data

,방류량,광진교_수위
0,555.00,1.20
1,562.90,1.20
2,576.40,1.20
3,563.10,1.20
4,576.40,1.20
...,...,...
276331,259.23,1.12
276332,260.46,1.12
276333,259.37,1.12
276334,259.13,1.12


In [6]:
x = raw_data

In [7]:
x

,방류량,광진교_수위
0,555.00,1.20
1,562.90,1.20
2,576.40,1.20
3,563.10,1.20
4,576.40,1.20
...,...,...
276331,259.23,1.12
276332,260.46,1.12
276333,259.37,1.12
276334,259.13,1.12


In [8]:
# 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

input_x_regular = x.copy()

for i in range(len(x.columns)):
    scaler = MinMaxScaler()                       #StandardScaler() , MinMaxScaler() , RobustScaler()
    a=np.array(input_x_regular.iloc[:,i])
    tmp_x = scaler.fit_transform(a.reshape(-1,1))
    input_x_regular.iloc[:,i] = tmp_x

# 원하는 칼럼명 입력하기

In [9]:
#ydata = raw_data[['현재수위','유입량','저수량','공용량','방류량','강화대교_조위','청담대교_유량','잠수교_유량','한강대교_유량']]
y = raw_data['방류량']

# 원하는 칼럼명 입력하기

In [10]:
input_regular = pd.concat([input_x_regular, y], axis = 1)
input_regular

,방류량,광진교_수위,방류량
0,0.057689,0.056452,555.00
1,0.057984,0.056452,562.90
2,0.058488,0.056452,576.40
3,0.057992,0.056452,563.10
4,0.058488,0.056452,576.40
...,...,...,...
276331,0.046645,0.045699,259.23
276332,0.046691,0.045699,260.46
276333,0.046650,0.045699,259.37
276334,0.046642,0.045699,259.13


# 원하는 칼럼명 입력하기

In [11]:
nanIndex = raw_data[raw_data['방류량'].isnull() == True].index

In [12]:
nanIndex

Int64Index([ 13200,  68081,  68083,  68084,  68085,  68086,  68087,  68089,
             68090,  68091,
            ...
            270533, 270534, 271588, 272215, 272216, 272217, 272218, 272219,
            272220, 272221],
           dtype='int64', length=743)

In [13]:
from tqdm import tqdm

xdata = []
ydata = []
xdata_idx = []
ydata_idx = [-1]
for i in range(len(input_regular.columns)-1):
    xdata_idx.append(i)

w =  6 #window size

idx = []
for i in range(12):
  idx.append(i*26496)

for i in tqdm(range(11)):
    if i != 10:
        tmp = input_regular[idx[i]:idx[i+1]]
    else:
        tmp = input_regular[idx[i]:]

    for j in range(len(tmp)-w):
        xdata.append(np.array(tmp.iloc[j:j + w, xdata_idx]).astype(float))

    for j in range(w,len(tmp)):
        ydata.append(np.array(tmp.iloc[j, ydata_idx]).astype(float))

100%|██████████| 11/11 [02:25<00:00, 13.22s/it]


In [31]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(275527, 6, 2)
(275527, 1)


In [15]:
# 나중에 학습된 모델에 이거 넣어서 predict해서 결측치 채우기
x_real_predict = []
y_real_predict = []

for i in range(len(nanIndex)):
    k = nanIndex[-(i+1)]
    
    x_real_predict.insert(0,(xdata[k - w * (k // 26496 + 1)]))
    y_real_predict.insert(0,(ydata[k - w * (k // 26496 + 1)]))
    
    xdata.pop(k - w * (k // 26496 + 1))
    ydata.pop(k - w * (k // 26496 + 1))

In [16]:
print(np.array(x_real_predict).shape)
print(np.array(y_real_predict).shape)

(743, 6, 2)
(743, 1)


In [17]:
print(np.array(xdata).shape)
print(np.array(ydata).shape)

(275527, 6, 2)
(275527, 1)


In [18]:
pd.DataFrame(ydata).isnull().sum()

0    0
dtype: int64

In [19]:
xtrain = np.array(xdata[:-50000])
ytrain = np.array(ydata[:-50000])

xtest = np.array(xdata[-50000:])
ytest = np.array(ydata[-50000:])

In [20]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

(225527, 6, 2)
(225527, 1)
(50000, 6, 2)
(50000, 1)


In [30]:
xtrain.shape[2]

2

## 모델 생성

In [26]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

if 'model_enc_dec_cnn' in locals(): 
    del model_enc_dec_cnn 
if 'history' in locals():
    del history

optimizer = tf.keras.optimizers.Adam(lr=0.01) #기본값 0.001

model_enc_dec_cnn = Sequential()
model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=2, padding = 'causal', activation='relu', input_shape=(w, xtrain.shape[2])))
#model_enc_dec_cnn.add(Conv1D(filters=64, kernel_size=2, padding = 'causal', activation='relu'))
#model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
model_enc_dec_cnn.add(Flatten())

model_enc_dec_cnn.add(RepeatVector(1))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
# model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(LSTM(200, activation='relu'))

model_enc_dec_cnn.add(Dense(100, activation='relu'))
model_enc_dec_cnn.add(Dense(1))

model_enc_dec_cnn.compile(loss=root_mean_squared_error, optimizer=optimizer, metrics =["accuracy"])

model_enc_dec_cnn.summary()

ValueError: ignored

In [27]:
# plot_model(model=model_enc_dec_cnn, show_shapes=True)

In [28]:
epochs = 100
batch_size = xtrain.shape[0] // 1000
validation = 0.2

checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', save_weights_only=False, monitor='val_loss',
                                         mode='min', save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=10, mode='min')
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.5, mode='min', patience=3, min_lr=0.001)

In [29]:
## 학습

history = model_enc_dec_cnn.fit(xtrain, ytrain, verbose=1, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])

RuntimeError: ignored

In [25]:
## 평가

keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

yhat = keras_model_best.predict(xtest, verbose=1)

yhat

keras_model_best.evaluate(xtest,ytest)

ValueError: ignored

In [ ]:
## 학습 과정 시각화

fig, loss_ax = plt.subplots(1, 2)
fig.set_size_inches((15,5))

loss_ax[0].plot(history.history['loss'], 'b', label='train loss')
loss_ax[0].plot(history.history['val_loss'], 'r', label='val loss')

loss_ax[1].plot(history.history['accuracy'], 'y', label='train acc')
loss_ax[1].plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax[0].set_xlabel('epoch')
loss_ax[0].set_ylabel('loss')
loss_ax[1].set_xlabel('epoch')
loss_ax[1].set_ylabel('accuray')

loss_ax[0].legend(loc='upper right')
loss_ax[1].legend(loc='lower right')

plt.show()

In [ ]:
## 결과 시각화

tl_predict = np.array(yhat)

#결과 시각화
plt.figure(figsize=(15,10))
plt.plot(ytest, marker = ",", label = "actual")
plt.plot(tl_predict, marker = ",", label="predict", alpha = 0.5)
plt.legend(prop={'size':10})  

plt.show()

In [ ]:
## 점수 확인
print("RMSE : ",float(root_mean_squared_error(ytest, tl_predict)))

## 파일 저장

In [ ]:
print(len(yhat))
print(len(nanIndex))

In [ ]:
x_real_predict = np.array(x_real_predict)
y_real_hat = keras_model_best.predict(x_real_predict, verbose = 1)

In [ ]:
root_mean_squared_error(y_real_predict, y_real_hat) #y_real_predict도 예측값밖에 없어서 의미없긴 하지만 얼마나 달라졌는지 보고 싶어서

In [ ]:
print(len(y_real_hat))
print(len(nanIndex))

In [ ]:
for i in range(len(nanIndex)):
    raw_data['방류량'].iloc[nanIndex[i]] = np.float64(y_real_hat[i])

In [ ]:
raw_data['방류량'].to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/newdata7_방류량_광진교_포함.csv', index = False, encoding = 'cp949')